# Census Case Study

 - Prepare the SQLAlchemy and the database
     1. Create and Engine and MetaData object
     2. Create and save the census table 
 - Loading Data into the Database
     3. Load a CSV file into a values list
     4. Insert the values list into the census table
 - Solving Data Science Problems with Queries

In [5]:
# Here goes all the imports
from sqlalchemy import create_engine, MetaData

from sqlalchemy import (Table, Column, String, 
                        Integer, DECIMAL, Boolean)

In [3]:
# Number 1
engine = create_engine('sqlite:///census.sqlite')

metadata = MetaData()

In [6]:
# Number 2 
employees = Table('employees', metadata,
                 Column('id', Integer()),
                 Column('name', String(255)),
                 Column('salary', DECIMAL()),
                 Column('active', Boolean())
                 )

metadata.create_all(engine)

In [7]:
# Import create_engine, MetaData
from sqlalchemy import create_engine, MetaData

# Define an engine to connect to chapter5.sqlite: engine
engine = create_engine('sqlite:///chapter5.sqlite')

# Initialize MetaData: metadata
metadata = MetaData()

In [8]:
# Import Table, Column, String, and Integer
from sqlalchemy import (Table, Column, String, Integer)

# Build a census table: census
census = Table('census', metadata,
               Column('state', String(30)),
               Column('sex', String(1)),
               Column('age', Integer()),
               Column('pop2000', Integer()),
               Column('pop2008', Integer()))

# Create the table in the database
metadata.create_all(engine)

# Populating the Database

In [9]:
# Number 3
values_list = []

for row in csv_reader:
    data = {'state':row[0], 'sex':row[1],
           'age':row[2], 'pop2000':row[3],
           'pop2008':row[4]}
    values_list.append(data)

# Number 4
from sqlalchemy import insert

stmt = insert(employees)

result_proxy = connection.execute(stmt, values_list)

print(result_proxy.rowcount)

NameError: name 'csv_reader' is not defined

# Querying the Database

1. Determine Average Age for Males and Females

In [13]:
from sqlalchemy import select, func

engine = create_engine('sqlite:///census.sqlite')
connection = engine.connect()

stmt = select([census.columns.sex,
              (func.sum(census.columns.pop2008 *
                       census.columns.age) / 
              func.sum(census.columns.pop2008)
              ).label('average_age')])

stmt = stmt.group_by(census.columns.sex)

results = connection.execute(stmt).fetchall()

2. Determine the percentage of Females for each state

In [14]:
from sqlalchemy import case, cast, Float

stmt = select([
    (func.sum(
    case([
        (census.columns.state == 'New York',
        census.columns.pop2008)
    ], else_=0)) / 
    cast(func.sum(census.columns.pop2008),
        Float) * 100).label('my_percent')
])

3. Determine the top 5 states by population change from 2000 to 2008

In [15]:
stmt = select([census.columns.age,
              (census.columns.pop2008 - 
              census.columns.pop2000
              ).label('pop_change')])

stmt = stmt.order_by('pop_change')

stmt = stmt.limit(5)

In [18]:
# Import select and func
from sqlalchemy import select, func

# Select the average of age weighted by pop2000
stmt = select([func.sum(census.columns.pop2000 * census.columns.age) / func.sum(census.columns.pop2000)])

# Relabel the new column as average_age
stmt = select([(func.sum(census.columns.pop2000 * census.columns.age) 
                / func.sum(census.columns.pop2000)).label('average_age')
              ])

# Add the sex column to the select statement
stmt = select([ census.columns.sex,
                (func.sum(census.columns.pop2000 * census.columns.age) 
                 / func.sum(census.columns.pop2000)).label('average_age'),               
              ])

# Group by sex
stmt = stmt.group_by(census.columns.sex)

# Import select and func
from sqlalchemy import select, func

# Select sex and average age weighted by 2000 population
stmt = select([(func.sum(census.columns.pop2000 * census.columns.age) 
                / func.sum(census.columns.pop2000)).label('average_age'),
               census.columns.sex])

# Group by sex
stmt = stmt.group_by(census.columns.sex)

# Execute the query and fetch all the results
results = connection.execute(stmt).fetchall()

# Print the sex and average age column for each result
for result in results:
    print(result.sex, result.average_age)

F 37
M 34


In [19]:
# import case, cast and Float from sqlalchemy
from sqlalchemy import cast, Float, case

# Build a query to calculate the percentage of women in 2000: stmt
stmt = select([census.columns.state,
    (func.sum(
        case([
            (census.columns.sex == 'F', census.columns.pop2000)
        ], else_=0)) /
     cast(func.sum(census.columns.pop2000), Float) * 100).label('percent_female')
])

# Group By state
stmt = stmt.group_by(census.columns.state)

# Execute the query and store the results: results
results = connection.execute(stmt).fetchall()

# Print the percentage
for result in results:
    print(result.state, result.percent_female)

Alabama 51.832407770179465
Alaska 49.301497893484594
Arizona 50.22361303057914
Arkansas 51.26992846221834
California 50.35233214901979
Colorado 49.84767060299562
Connecticut 51.66816507130644
Delaware 51.61109733558627
District of Columbia 53.129626141738385
Florida 51.36488001165242
Georgia 51.11408350339436
Hawaii 51.118011836915514
Idaho 49.98972623903102
Illinois 51.11224234802867
Indiana 50.95480313297678
Iowa 50.950398342534264
Kansas 50.821864107754735
Kentucky 51.32687036927168
Louisiana 51.75351596554121
Maine 51.50570813418951
Maryland 51.93575549972231
Massachusetts 51.843023571316785
Michigan 50.97246518318712
Minnesota 50.49332944301148
Mississippi 51.92229481794672
Missouri 51.46888602639692
Montana 50.32202690728538
Nebraska 50.8584549336086
Nevada 49.36736361384359
New Hampshire 50.858019844961746
New Jersey 51.51713956125773
New Mexico 51.0471720798335
New York 51.83453865150073
North Carolina 51.482262322084594
North Dakota 50.50069363231332
Ohio 51.46550350015544
Okl

In [21]:
# Import
from sqlalchemy import desc

# Build query to return state name and population difference from 2008 to 2000
stmt = select([census.columns.state,
     (census.columns.pop2008-census.columns.pop2000).label('pop_change')
])

# Group by State
stmt = stmt.group_by(census.columns.state)

# Order by Population Change
stmt = stmt.order_by(desc('pop_change'))

# Limit to top 10
stmt = stmt.limit(10)

# Use connection to execute the statement and fetch all results
results = connection.execute(stmt).fetchall()

# Print the state and population change for each record
for result in results:
    print('{}:{}'.format(result.state, result.pop_change))

Texas:40137
California:35406
Florida:21954
Arizona:14377
Georgia:13357
North Carolina:11574
Virginia:6639
Colorado:6425
Utah:5934
Illinois:5412
